In [1]:
# =========================================
# Inteligencia Artificial - Tarea práctica
# Conceptos: Estado, Espacio de Estados, Acciones, Recompensa y Ambiente
# =========================================

# ========================
# 1. VARIABLES DE ESTADO
# ========================
estado_robot = {
    "posicion": (0, 0),
    "bateria": 100,
    "objetivo_alcanzado": False
}

print("Estado inicial del robot:", estado_robot)

# ========================
# 2. ESPACIO DE ESTADOS
# ========================
posiciones = [(x, y) for x in range(3) for y in range(3)]
baterias = range(0, 101, 10)  # Batería de 0 a 100 en pasos de 10

espacio_estados = [(p, b) for p in posiciones for b in baterias]
print("\nTotal de estados posibles:", len(espacio_estados))
print("Ejemplos de estados:", espacio_estados[:5])

# ========================
# 3. ESPACIO DE ACCIONES
# ========================
acciones = ["adelante", "atras", "izquierda", "derecha", "recargar"]
print("\nAcciones posibles:", acciones)

# ========================
# 4. FUNCIÓN DE RECOMPENSA
# ========================
def recompensa(accion, estado, paso, objetivo_alcanzado_en_menos_de_5):
    # Castigo por moverse sin batería
    if estado["bateria"] <= 0 and accion in ["adelante", "atras", "izquierda", "derecha"]:
        return -5

    # Bonus por recargar
    if accion == "recargar":
        return 5

    # Bonus por alcanzar objetivo rápido
    if objetivo_alcanzado_en_menos_de_5 and estado["objetivo_alcanzado"]:
        return 20

    # Recompensa por alcanzar objetivo
    if estado["objetivo_alcanzado"]:
        return 10

    # Costo de movimiento normal
    if accion in ["adelante", "atras", "izquierda", "derecha"]:
        return -1

    return 0

# ========================
# 5. AMBIENTE Y SIMULACIÓN
# ========================
import random

def mover_robot(estado, accion):
    x, y = estado["posicion"]
    bateria = estado["bateria"]

    # Verificar si puede realizar la acción
    if bateria <= 0 and accion != "recargar":
        return estado  # No puede moverse sin batería

    # Consumo de batería (excepto para recargar)
    if accion in ["adelante", "atras", "izquierda", "derecha"]:
        estado["bateria"] = max(0, bateria - 10)

    # Ejecutar movimiento
    if accion == "adelante" and bateria > 0:
        x = min(x + 1, 2)
    elif accion == "atras" and bateria > 0:
        x = max(x - 1, 0)
    elif accion == "derecha" and bateria > 0:
        y = min(y + 1, 2)
    elif accion == "izquierda" and bateria > 0:
        y = max(y - 1, 0)
    elif accion == "recargar":
        estado["bateria"] = 100  # Recarga completa

    estado["posicion"] = (x, y)

    # Verificar objetivo
    if estado["posicion"] == (2, 2):
        estado["objetivo_alcanzado"] = True

    return estado

# ========================
# 6. ESTRATEGIAS DE MOVIMIENTO
# ========================
def estrategia_aleatoria(estado):
    return random.choice(acciones)

def estrategia_conservadora(estado):
    x, y = estado["posicion"]

    # Prioridad 1: Recargar si batería baja
    if estado["bateria"] < 30 and estado["posicion"] == (0, 0):
        return "recargar"

    # Prioridad 2: Moverse hacia el objetivo (2,2)
    if x < 2:
        return "adelante"
    if y < 2:
        return "derecha"

    return random.choice(acciones)

def estrategia_agresiva(estado):
    # Solo recarga cuando batería está totalmente agotada
    if estado["bateria"] == 0:
        return "recargar"

    # Movimiento directo al objetivo
    x, y = estado["posicion"]
    if x < 2:
        return "adelante"
    if y < 2:
        return "derecha"

    return "recargar"  # Si ya está en objetivo

# ========================
# 7. SIMULACIÓN COMPARATIVA
# ========================
def simular_robot(estrategia, max_pasos=10):
    estado = {"posicion": (0, 0), "bateria": 100, "objetivo_alcanzado": False}
    recompensa_total = 0
    objetivo_alcanzado_en_menos_de_5 = False

    print(f"\n\n=== Estrategia: {estrategia.__name__} ===")
    print("Paso 0: Estado inicial:", estado)

    for paso in range(1, max_pasos + 1):
        accion = estrategia(estado)
        estado_anterior = estado.copy()
        estado = mover_robot(estado, accion)

        # Verificar si llegó rápido al objetivo
        if paso < 5 and estado["objetivo_alcanzado"]:
            objetivo_alcanzado_en_menos_de_5 = True

        r = recompensa(accion, estado, paso, objetivo_alcanzado_en_menos_de_5)
        recompensa_total += r

        print(f"Paso {paso}: Acción = {accion:>9} | Posición: {estado_anterior['posicion']} -> {estado['posicion']} | "
              f"Batería: {estado_anterior['bateria']} -> {estado['bateria']} | "
              f"Objetivo: {estado['objetivo_alcanzado']} | Recompensa: {r:3}")

        if estado["objetivo_alcanzado"]:
            break

    print(f"\nRecompensa total: {recompensa_total}")
    print(f"Objetivo alcanzado: {'Sí' if estado['objetivo_alcanzado'] else 'No'}")
    print(f"Batería final: {estado['bateria']}%")
    return recompensa_total

# Ejecutar simulaciones con diferentes estrategias
resultados = {
    "Aleatoria": simular_robot(estrategia_aleatoria),
    "Conservadora": simular_robot(estrategia_conservadora),
    "Agresiva": simular_robot(estrategia_agresiva)
}

# Mostrar comparación final
print("\n\n=== COMPARACIÓN DE ESTRATEGIAS ===")
for estrategia, recompensa in resultados.items():
    print(f"{estrategia:>12}: {recompensa} puntos")

Estado inicial del robot: {'posicion': (0, 0), 'bateria': 100, 'objetivo_alcanzado': False}

Total de estados posibles: 99
Ejemplos de estados: [((0, 0), 0), ((0, 0), 10), ((0, 0), 20), ((0, 0), 30), ((0, 0), 40)]

Acciones posibles: ['adelante', 'atras', 'izquierda', 'derecha', 'recargar']


=== Estrategia: estrategia_aleatoria ===
Paso 0: Estado inicial: {'posicion': (0, 0), 'bateria': 100, 'objetivo_alcanzado': False}
Paso 1: Acción =  adelante | Posición: (0, 0) -> (1, 0) | Batería: 100 -> 90 | Objetivo: False | Recompensa:  -1
Paso 2: Acción =  adelante | Posición: (1, 0) -> (2, 0) | Batería: 90 -> 80 | Objetivo: False | Recompensa:  -1
Paso 3: Acción =     atras | Posición: (2, 0) -> (1, 0) | Batería: 80 -> 70 | Objetivo: False | Recompensa:  -1
Paso 4: Acción =     atras | Posición: (1, 0) -> (0, 0) | Batería: 70 -> 60 | Objetivo: False | Recompensa:  -1
Paso 5: Acción = izquierda | Posición: (0, 0) -> (0, 0) | Batería: 60 -> 50 | Objetivo: False | Recompensa:  -1
Paso 6: Acción